# <font color='blue'> The Loop Converter Package (loop.py) </font>
<p>
<center>
<img width=700 src="https://humanities-web.s3.us-east-2.amazonaws.com/college/chicagostudies-prod/s3fs-public/2020-08/B%26W%20Chicago%20Map.jpg"></img>

In [7]:

from geopy.geocoders import Nominatim

def get_coordinates(address):
    geolocator = Nominatim(user_agent="address_converter")
    location = geolocator.geocode(address)
    
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

# Example usage
address = "5500 South State Street, Chicago, IL"
coordinates = get_coordinates(address)

if coordinates:
    latitude, longitude = coordinates
    print(f"Latitude: {latitude}")
    print(f"Longitude: {longitude}")
else:
    print("Invalid address or could not find coordinates.")
    
# Baseline
b_latitude, b_longitude = 41.88205727768228, -87.62783047240069
"""b_address = "6 E Madison St, Chicago, IL 60602"
coordinates = get_coordinates(b_address)

if coordinates:
    b_latitude, b_longitude = coordinates
    print(f"Latitude of Baseline: {b_latitude}")
    print(f"Longitude of Baseline: {b_longitude}")
else:
    print("Invalid address or could not find coordinates.")"""

Latitude: 41.7941402
Longitude: -87.6259778


'b_address = "6 E Madison St, Chicago, IL 60602"\ncoordinates = get_coordinates(b_address)\n\nif coordinates:\n    b_latitude, b_longitude = coordinates\n    print(f"Latitude of Baseline: {b_latitude}")\n    print(f"Longitude of Baseline: {b_longitude}")\nelse:\n    print("Invalid address or could not find coordinates.")'

In [8]:
#miles per degree

mpd = (24901.92/360)

#blocks per degree
bpd_n_mr = mpd*12
bpd_n_rc = mpd*10
bpd_n_ct = mpd*9
bpd_n = mpd*8
bpd_w = mpd*8

roosevelt = 41.8674224
cermak = 41.8528516
thirtyfirst = 41.8382902
#Finding the blocks of our address
if latitude > b_latitude:
    print((latitude-b_latitude)*bpd_n,"blocks north")
elif roosevelt <= latitude <= b_latitude:
    print((b_latitude-latitude)*bpd_n_mr)
elif cermak <= latitude <= roosevelt:
    print(((b_latitude-roosevelt)*bpd_n_mr+(roosevelt-latitude)*bpd_n_rc))
elif thirtyfirst <= latitude <= cermak:
    print((((b_latitude-roosevelt)*bpd_n_mr+(roosevelt-cermak)*bpd_n_rc+(cermak-latitude)*bpd_n_ct)))
elif latitude <= thirtyfirst:
    print((((b_latitude-roosevelt)*bpd_n_mr+(roosevelt-cermak)*bpd_n_rc+(cermak-thirtyfirst)*bpd_n_ct+(thirtyfirst-latitude)*bpd_n)))

55.723519731661696


In [9]:
if longitude >= b_longitude:
    print(abs(b_longitude-longitude)*bpd_w,"blocks east")
elif longitude <= b_longitude:
    print(abs(longitude-b_longitude)*bpd_w,"blocks west")

1.0252244424028671 blocks east


In [8]:
print(b_longitude)

-87.62783047240069


In [9]:
#miles per degree

mpd = (24901.92/360)

#blocks per degree
bpd_n_mr = mpd*12
bpd_n_rc = mpd*10
bpd_n_ct = mpd*9
bpd_n = mpd*8
bpd_w = mpd*8

print(bpd_n)


553.376
